<a href="https://colab.research.google.com/github/sachinmasti/custom-scalers-from-scratch/blob/main/train_test_splitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd
import numpy as np
import seaborn as sns

In [51]:
from sklearn.model_selection import  train_test_split

In [52]:
df = sns.load_dataset('titanic')

In [53]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [54]:
df = df.drop(columns=['deck', 'survived', 'alone', 'adult_male'])

In [55]:
df.head()

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,embark_town,alive
0,3,male,22.0,1,0,7.2500,S,Third,man,Southampton,no
1,1,female,38.0,1,0,71.2833,C,First,woman,Cherbourg,yes
2,3,female,26.0,0,0,7.9250,S,Third,woman,Southampton,yes
3,1,female,35.0,1,0,53.1000,S,First,woman,Southampton,yes
4,3,male,35.0,0,0,8.0500,S,Third,man,Southampton,no


In [67]:
df.isna().sum()

,0
pclass,0
sex,0
age,177
sibsp,0
parch,0
fare,0
embarked,2
class,0
who,0
embark_town,2


In [56]:
split = train_test_split

In [60]:
x_train,x_test,y_train,y_test = split(df.drop(columns='alive'),df['alive'],random_state=42,shuffle=True,test_size=0.2)

In [108]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline

# Group columns by type
cat_cols = ['sex', 'embarked', 'class', 'who', 'embark_town']
num_cols = ['pclass', 'age', 'sibsp', 'parch', 'fare']

# Create preprocessing steps
# Note: We can nest an imputer inside a Pipeline for the categorical columns
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

col_trans = ColumnTransformer(transformers=[
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, cat_cols)
], remainder='drop')

In [106]:
col_trans

ColumnTransformer(transformers=[('age_impute', SimpleImputer(), ['age']),
                                ('town_impute',
                                 SimpleImputer(strategy='most_frequent'),
                                 ['embark_town']),
                                ('encode',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['sex', 'embarked', 'class', 'who',
                                  'embark_town']),
                                ('scale', StandardScaler(),
                                 ['pclass', 'age', 'sibsp', 'parch', 'fare'])])

In [94]:
from sklearn.pipeline import Pipeline,make_pipeline

In [95]:
pipe = make_pipeline(col_trans,DecisionTreeClassifier())

In [110]:
from sklearn.tree import DecisionTreeClassifier
# Re-defining the pipe to ensure it uses the updated col_trans
pipe = make_pipeline(col_trans, DecisionTreeClassifier())
pipe.fit(x_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'sibsp',
                                                   'parch', 'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sex', 'embarked', 'class',
                                                   'who', 'embark_town'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [109]:
from sklearn import  set_config
set_config(display='diagram')

In [111]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pipe.predict(x_test))

0.770949720670391